**An analytical permafrost model** (Kudryavtsev's model) see (Sazonova et al., 2003. DOI: 10.1002/ppp.449, http://onlinelibrary.wiley.com/doi/10.1002/ppp.449/abstract). Author: Kang Wang. Date: March 3, 2016


In [1]:
import numpy as np

In [2]:
# INPUTs :
#   Aa: amplitude of Tair
#   Ta: annual mean air temperature
#   Hsn: winter-averaged snow thickness
#   rho_sn: snow density
#   W_vol: volumetric water content
#   Hvg1: winter-averaged height of vegetation cover
#   Hvg2: summer-averaged height of vegetation cover
#   Dvf: thermal conductivity of vegation in frozen state
#   Dvt: thermal conductivity of vegation in thaw state
#   Soil Texture, including percent of sand, silt and clay
#   Kf: thermal conductivity of frozen ground
#   Kt: thermal conductivity of thawed ground
#   Ct: volumetric heat capacity of thawed ground
#   Cf: volumetric heat capacity of frozen ground

rho_sn  = 240.;
Hvg1    = 0.0;
Hvg2    = 0.0;
Dvf     = 1.39E-6;
Dvt     = 5.56E-8;
#Kf      = 2.305;
#Kt      = 1.865;

# Here we used measurements from http://permafrost.gi.alaska.edu for Barrow, AK site, 2012 
Ta = -10.81
Aa = 19.04
Hsn = 0.28
W_vol = 0.41

# We use soil properties for Barrow site from the modified Harmonizied Soil Database. 
# Global Soil Texture and derived water holding capacities 
# http://webmap.ornl.gov/ogcdown/wcsdown.jsp?dg_id=548_6

percent_silt = 0.30
percent_sand = 0.60
percent_clay = 0.10

Bulk_Density_Silt = 1400.
Heat_Capacity_Silt = 730.

Bulk_Density_Sand = 1300.
Heat_Capacity_Sand = 690.

Bulk_Density_Clay = 1500.
Heat_Capacity_Clay = 900.

# Wet soils 
Kf_Silt = 2.40
Kt_Silt = 1.90

Kf_Sand = 2.65
Kt_Sand = 2.15

Kf_Clay = 2.00
Kt_Clay = 1.70

# Dry soils
Kf_Silt = (1.25 + Kf_Silt) / 2.
Kt_Silt = (1.05 + Kt_Silt) / 2.

Kf_Sand = (1.25 + Kf_Sand) / 2.
Kt_Sand = (1.05 + Kt_Sand) / 2.

Kf_Clay = (1.15 + Kf_Clay) / 2.
Kt_Clay = (0.90 + Kt_Clay) / 2.

Heat_Capacity = Heat_Capacity_Clay**percent_clay * \
                Heat_Capacity_Sand**percent_sand * \
                Heat_Capacity_Silt**percent_silt
                
Bulk_Density =  Bulk_Density_Clay**percent_clay * \
                Bulk_Density_Sand**percent_sand * \
                Bulk_Density_Silt**percent_silt

Here we estimate volumetric heat capacities (Ct and Cf); and thermal conductivities (Kt and Kf) based on  Anisimov et al., (1997) (doi:10.1016/S0921-8181(97)00009-X, http://www.sciencedirect.com/science/article/pii/S092181819700009X).

In [3]:
# Anisimov et al. (1997)
Ct =Heat_Capacity*Bulk_Density + 4190.*W_vol;
Cf =Heat_Capacity*Bulk_Density + 2025.*W_vol;

Kt = Kt_Silt**percent_silt * Kt_Clay**percent_clay * Kt_Sand**percent_sand
Kf = Kf_Silt**percent_silt * Kf_Clay**percent_clay * Kf_Sand**percent_sand

# Estimate Ksn and Csn from rho_sn 

Ksn = (rho_sn/1000.)*(rho_sn/1000.)*3.233-1.01*(rho_sn/1000)+0.138;
Csn = rho_sn*2.09E3;

tao = 365.*24.*3600.;
tao1 = tao*(0.5 - 1./np.pi*np.arcsin(Ta/Aa)); # Cold Season 
tao2 = tao - tao1; # Warm Season
    
L = (3.35E8*W_vol*1.); # Latent Heat

Estimation of the potential **snow** and **vegetation** effects. 

In [4]:
# Snow 
    
alpha = 2.*Aa*Cf/L;
beta  = 2.*abs(Ta)*Cf/L;

Cef = Cf*((alpha-beta)/((alpha - beta) - np.log((alpha+1.)/(beta+1.))));

mu = (np.sqrt(Ksn*Csn) - np.sqrt(Kf*Cef))\
    /(np.sqrt(Ksn*Csn) + np.sqrt(Kf*Cef));

s = np.exp(2.*Hsn*np.sqrt(np.pi*Csn/(tao*Ksn)))\
    +2.*mu*np.cos(2.*Hsn*np.sqrt(np.pi*Csn/(tao*Ksn)))\
    +mu**2.*np.exp(-2.*Hsn*np.sqrt(np.pi*Csn/(tao*Ksn)));

deta_A = Aa*(1.0- (1.+mu)/np.sqrt(s));
deta_Asn = deta_A*tao1/tao*0.5;
deta_Tsn = 2./np.pi*deta_Asn;

# Vegetation

Tvg = Ta + deta_Tsn;
Avg = Aa - deta_Asn;

deta_A1 = (Avg - abs(Tvg)) * (1.-np.exp(-1.*Hvg1*np.sqrt(np.pi/(Dvf*2.*tao1))));
deta_A2 = (Aa  - abs(Ta) ) * (1.-np.exp(-1.*Hvg2*np.sqrt(np.pi/(Dvt*2.*tao2))));

deta_Av = (deta_A1*tao1+deta_A2*tao2) / tao;
deta_Tv = (deta_A1*tao1-deta_A2*tao2) / tao * (2. / np.pi);

Tgs = Tvg+deta_Tv;
Ags = Avg-deta_Av;

Calculation of the mean annual **temperature at the top of permafrost (TTOP)** and the **active layer thickness (ALT)**

In [5]:
Tps_numerator = 0.5*Tgs*(Kf+Kt)\
                +(Ags*(Kt-Kf)/np.pi\
                *(Tgs/Ags*np.arcsin(Tgs/Ags)\
                +np.sqrt(1.-(np.pi*np.pi/(Ags*Ags)))));

if Tps_numerator<=0.0:
    K_star = Kf;
    C = Cf;
    K = Kf;
    Flag = -1;
else:
    K_star = Kt;
    C = Ct;
    K = Kt;
    Flag = 1;

Tps = Tps_numerator/K_star;  # Temperature at the top of permafrost


Aps = ((Ags - abs(Tps))/np.log((Ags+L/(2.*C))/(abs(Tps)+L/(2.*C))))- L/(2.*C);

Zc = (2*(Ags - abs(Tps))*np.sqrt((K*tao*C)/np.pi)) / (2.*Aps*C + L);

Zal = (2*(Ags - abs(Tps))*np.sqrt(K*tao*C/np.pi)\
    +(((2.*Aps*C*Zc+L*Zc)*L*np.sqrt(K*tao/(np.pi*C)))\
    /(2.*Ags*C*Zc + L*Zc +(2.*Aps*C+L)*np.sqrt(K*tao/(np.pi*C)))))\
    /(2.*Aps*C+L);  # Active Layer Thickness

print 'TTOP=',Tps

print 'ALT=',Zal

TTOP= -7.69149521078
ALT= 0.578469574333
